In [2]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm, rgb2hex

DO_CLUSTERING = False
communities = [{3, 7, 1}, {2, 5}, {0, 4, 6}]
edges = {(3, 7, 2), (1, 7, 5), (1, 2, 1), (0, 4, 5), (0, 6, 5), (2, 5, 7)}
coordinates = [[48.090833 ,13.877222], [47.766943,12.457000], [43.928889,2.146389], [50.883333,20.616667], [47.1351,15.2851], [50.0875,14.421389], [45.466667 ,9.166667], [52.21962,-3.93517]]
sizes_of_vertices = [2, 3, 7, 6, 5, 5, 2, 4]
centuries_of_points = [12, 11, 14, 15, 12, 13, 13, 11]
map_centre = [47.466667, 11.166667] # Centre of returned maps

map = folium.Map(map_centre, zoom_start=5, height='70%', width='70%', tiles=None)

#tile_layer = folium.TileLayer(
#    tiles="https://{s}.basemaps.cartocdn.com/rastertiles/dark_all/{z}/{x}/{y}.png",
#    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
#    max_zoom=19,
#    name='darkmatter',
#    control=False,
#    opacity=0.3)
# because we do not wanna have 'openstreetmap' in LayerControl
tile_layer = folium.TileLayer(control=False)
tile_layer.add_to(map)

cluster = MarkerCluster(name="Clustering of points", control=False).add_to(map) 

cmap = plt.get_cmap('plasma')
offset = TwoSlopeNorm(vmin = 0, vcenter= (len(communities) / 2), vmax = len(communities))
colors = []
for b in range(len(communities)):
    scale = offset(b)
    color=rgb2hex(cmap(scale))
    colors.append(color)

community_layers = {}
century_layers = {}
for century in centuries_of_points:
    c_layer = folium.FeatureGroup(name=str(century)+". century", show=False)
    century_layers[century] = c_layer
    map.add_child(c_layer)

# Points
i, j = 0, 0
for community in communities:
    com = folium.FeatureGroup(name="Community "+str(i+1), show=True)
    for source in community:
        cen = century_layers[centuries_of_points[source]]
        community_layers[source] = (com, i)
        info = "<h4> Point "+str(source) + "</h4> <h5> Size: "+str(sizes_of_vertices[source])+"</h5> <h5> Century: "+str(centuries_of_points[source])+". century </h5>"
        popup1 = folium.Popup(info, max_width=300, min_width =300)
        popup2 = folium.Popup(info, max_width=300, min_width =300)
        lat = coordinates[source][0]
        long = coordinates[source][1]
                #place point on map (variant cluster.add ... if not in comment above)
        if DO_CLUSTERING:
            cluster.add_child(folium.CircleMarker(location=[lat , long], fill=True, color = colors[i],fill_opacity=0.4, popup=popup1))
        else:
            #folium.CircleMarker(location=[lat , long], fill=True, color = colours[i],fill_opacity=0.4, popup=popup).add_to(folium.FeatureGroup(name='M').add_to(map))
            #map.add_child(folium.CircleMarker(location=[lat , long], fill=True, color = colours[i],fill_opacity=0.4, popup=popup, radius = sizes_of_vertices[source] * 3))
            folium.CircleMarker(location=[lat , long], fill=True, color = colors[i],
                                fill_opacity=0.4, popup=popup1, radius = sizes_of_vertices[source] * 3).add_to(com)
            folium.CircleMarker(location=[lat , long], fill=True, color = 'gray',
                                fill_opacity=0.6, popup=popup2, radius = sizes_of_vertices[source] * 3).add_to(cen)
        j += 1
    map.add_child(com)
    i += 1


# Lines
for line in edges:
    popup1 = folium.Popup('<h5> Jaccard distance: ' + str(line[2]) + '</h5>', max_width=300)
    popup2 = folium.Popup('<h5> Jaccard distance: ' + str(line[2]) + '</h5>', max_width=300)
    # once popups are shared (it is only one here) it does not work
    if community_layers[line[0]][1] == community_layers[line[1]][1]:
        color = colors[community_layers[line[0]][1]]
        com = community_layers[line[0]][0]
        folium.PolyLine(locations=[coordinates[line[0]], coordinates[line[1]]], weight = line[2], popup=popup1, color = color).add_to(com)
    
    else:
        color = 'black'
        com1 = community_layers[line[0]][0]
        com2 = community_layers[line[1]][0]
        folium.PolyLine(locations=[coordinates[line[0]], coordinates[line[1]]], weight = line[2], popup=popup1, color = color).add_to(com1)
        folium.PolyLine(locations=[coordinates[line[0]], coordinates[line[1]]], weight = line[2], popup=popup2, color = color).add_to(com2)


folium.LayerControl(collapsed=False).add_to(map)           

map.save('map_experiment.html')

neshoda
[47.766943, 12.457]
shoda 1
#9c179e
shoda 2
#ed7953
shoda 0
#0d0887
shoda 0
#0d0887
shoda 2
#ed7953


Notes:  
- layer with switch for points works, but I cannot see any way of making lines/edges part of this
- it seems to me, especially when we think about table/chart view as well, that  
- 
- FeatureGroups - visualizations logic is OR, means that shared line dissapears once both communities are switched off
- Centuries filtering is done via colour change of chosen century to gray and leaving edges as they are - bearing the info about communities